In [ ]:
from fins3666.defines import *
from fins3666.utils import format_market_data, current_fx_data
from fins3666.portfolio import Portfolio
from fins3666.models.carry_trade import generate_orders

startTime = datetime(2001, 1, 1)

dates = mkt.loc[mkt['Timestamp'] >= startTime, 'Timestamp']
endTime = dates.iloc[-1]


fx_start = pd.DataFrame(current_fx_data(fx, startTime))


startingAssets = [
    {'asset': 'USD', 'units': 0, 'unit_value_USD': 1.0}]

date = startTime
currentData = current_fx_data(fx, date)

SamplePortfolio = Portfolio(startTime, startingAssets, currentData)


print(SamplePortfolio)
account_reference = ACCOUNT_SIZE_USD
prev_month = startTime.month

for date in dates:
    month = date.month
    yields = mkt[mkt['Timestamp'] <= date]

    fx_rates = current_fx_data(fx, date)

    SamplePortfolio.updateMarketData(
        currentData=fx_rates, yields=yields)

    # Loop that runs once per month
    if (month != prev_month):
        prev_month = month
        # determine which assets to long buy and which to short
        current_positions = SamplePortfolio.summary()
        fx_arrays = SamplePortfolio.forex_spreads

        newOrders = generate_orders(
            yields=yields,
            current_positions=SamplePortfolio.summary(),
            fx=fx_arrays,
            size=account_reference)

        for order in newOrders:
            SamplePortfolio.new_order(order)

        print(SamplePortfolio)

        account_reference = index_inflation(ACCOUNT_SIZE_USD, date)
    SamplePortfolio.update(date)


print(SamplePortfolio)



                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00             0     $              0.00             Long           0.00%         nan%

Portfolio - 1 assets
Timestamp: 2001-01-01 00:00:00,
Net Value ($USD): $0.00


                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00             0     $              0.00             Long           0.00%         nan%

Portfolio - 1 assets
Timestamp: 2001-01-31 00:00:00,
Net Value ($USD): $0.00


                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00             0     $   

In [4]:
startTime = datetime(2024, 6, 28)
startingAssets = [{
    'asset': 'USD',
    'units': 50_000,
    'unit_value_USD': 1.0},
    {
    'asset': 'NOK',
        'units': 10_000,
        'unit_value_USD': 0.094
}]
currentData = [{
    'timestamp': startTime,
    'asset': "NOK",
    'tic': "USD/NOK",
    'bid': 10.6446,
    'ask': 10.6504,
    'mid': 10.6475,
}]

SamplePortfolio = Portfolio(startTime, startingAssets, currentData)
print(SamplePortfolio)



                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00        50,000     $         50,000.00             Long           0.00%       98.16%
NOK      $0.09        10,000     $            939.19             Long           0.00%        1.84%

Portfolio - 2 assets
Timestamp: 2024-06-28 00:00:00,
Net Value ($USD): $50,939.19


In [ ]:
order_example = Order(
    asset="NOK/USD",
    units=100_000,
    order="buy",
    timestamp=datetime(2024, 6, 28),
    order_type="limit",
    expiry="2025-06-01T12:00:00",
    limit=0.09391,
    exchange="NYSE"
)
print(order_example)
print('\n')
print(order_example.summary_str())

Order ID: f0661a90-28f7-11f0-b460-00155dd4b9c5 | Asset: NOK/USD | Units: 100000 | Order Type: limit | Order Side: buy | Timestamp: 2024-06-28T00:00:00 | Currency: USD | Limit Price: 0.09391 | Expiry: 2025-06-01T12:00:00 | Exchange: NYSE | Asset Type: Currency


BUY 100000 NOK/USD @ 0.09391 (ID: f0661a90)


In [ ]:

MARGIN = 1.2e6

balance = SamplePortfolio.balance + MARGIN

benchmark = benchmarks.loc[benchmarks['Timestamp'].isin(
    balance.index), ['SP500', 'Timestamp']]

benchmark = pd.Series(benchmark['SP500'].tolist(),
                      index=benchmark['Timestamp'].tolist())/100+1

benchmark = benchmark*MARGIN


# Ensure the comparison is done for each timestamp in benchmarks
rf_rate = mkt.loc[mkt['Timestamp'].isin(
    balance.index), ['USD LONG', 'Timestamp']]

rf_rate_pa = pd.Series(
    (rf_rate['USD LONG']/100).tolist(), index=rf_rate['Timestamp'])

test = Portfolio.metrics(balance, benchmark, rf_rate_pa)
print(f'Sharpe Ratio: {test['Sharpe Ratio']}')
print(f'Sortino Ratio: {test['Sortino Ratio']}')
print(f'Volatility: {test['Volatility']}')
print(f'Alpha: {test['Alpha']}')
print(f'Beta: {test['Beta']}')
print(f'Max Drawdown: {test['Max Drawdown']}')
print(f'Beta: {test['Beta']}')


test['df'].to_csv('./data/processed/results.csv')